In [1]:
import os
os.chdir('../../../..')

In [2]:
import convokit

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
from convokit import Forecaster, Corpus, download

In [4]:
MAX_LENGTH = 80

In [5]:
craft_model = convokit.CRAFTModel(device_type="cpu", options={'validation_size': 0.2,
                                                             })

Loading saved parameters...
Building encoders, decoder, and classifier...
Models built and ready to go!


In [6]:
forecaster = Forecaster(forecaster_model = craft_model,
                        forecast_mode = 'past',
                        convo_structure="linear",
                        text_func = lambda utt: utt.meta["tokens"][:(MAX_LENGTH-1)],
                        utt_selector_func = lambda utt: not utt.meta["is_section_header"],
                        label_func = lambda utt: int(utt.meta['comment_has_personal_attack']),
                        convo_selector_func = (lambda convo: convo.meta["split"] == "train"),
                        forecast_feat_name="prediction", forecast_prob_feat_name="pred_score",
                        use_last_only = True,
                        skip_broken_convos=False
                       )

In [7]:
corpus = Corpus(filename=download("conversations-gone-awry-corpus"))

Dataset already exists at /Users/calebchiam/.convokit/downloads/conversations-gone-awry-corpus


In [8]:
from convokit import craft_tokenize

In [9]:
for utt in corpus.iter_utterances():
    utt.add_meta("tokens", craft_tokenize(craft_model.voc, utt.text))

In [ ]:
forecaster.fit(corpus)

Building optimizers...
Starting Training!
Will train for 930 iterations
Initializing ...
Training...
Iteration: 10; Percent complete: 1.1%; Average loss: 0.3532
Iteration: 20; Percent complete: 2.2%; Average loss: 0.3778


In [ ]:
forecaster.forecast_mode = "past"

In [ ]:
forecaster.transform(corpus)